# Covid_19 Analysis - Part 1 (Data Preparation)

## Objective

To create an interactive visual comparison of CovId19 infections between different countries

## Dataset Information

The dataset was acquired from HDX (HUMANITARIAN DATA EXCHANGE) website.
https://data.humdata.org/dataset/novel-coronavirus-2019-ncov-cases

Our World in Data - https://github.com/owid/covid-19-data/tree/master/public/data
data dictionary - https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-codebook.csv

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import datetime

In [2]:
import warnings
warnings.filterwarnings(action='once')

In [3]:
import cufflinks as cf
import plotly.offline as pyo

C:\Users\Susan\Anaconda3\lib\site-packages\nbformat\notebooknode.py:4: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working

C:\Users\Susan\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

C:\Users\Susan\Anaconda3\lib\site-packages\pywt\_utils.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working



In [4]:

from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [5]:
!dir D:\datasets\covid

 Volume in drive D is New Volume
 Volume Serial Number is AC49-1552

 Directory of D:\datasets\covid

16/11/2020  11:22 AM    <DIR>          .
16/11/2020  11:22 AM    <DIR>          ..
16/11/2020  11:07 PM        22,414,537 covid_processed.csv
12/11/2020  11:03 PM        13,063,277 owid-covid-data.csv
13/11/2020  01:40 PM        47,057,658 owid-covid-data.json
12/11/2020  10:15 PM           353,769 time_series_covid19_confirmed_global_iso3_regions.csv
12/11/2020  10:11 PM         4,658,347 time_series_covid19_confirmed_global_narrow.csv
12/11/2020  10:27 PM         4,555,558 time_series_covid19_deaths_global_narrow.csv
13/11/2020  02:05 PM           307,038 time_series_covid19_recovered_global.csv
12/11/2020  10:27 PM         4,391,268 time_series_covid19_recovered_global_narrow.csv
               8 File(s)     96,801,452 bytes
               2 Dir(s)  754,904,461,312 bytes free


C:\Users\Susan\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning:

unclosed file <_io.BufferedWriter name=4>

C:\Users\Susan\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning:

unclosed file <_io.BufferedReader name=5>

C:\Users\Susan\Anaconda3\lib\site-packages\IPython\utils\_process_win32.py:131: ResourceWarning:

unclosed file <_io.BufferedReader name=6>



In [6]:
pd.set_option('display.max_columns', None)

In [7]:
path = "D:\datasets\covid"

<>:1: DeprecationWarning:

invalid escape sequence \d



Firstly read the file "time_series_covid19_confirmed_global_iso3_regions.csv", to get some general country attributes

In [8]:
df_general = pd.read_csv(os.path.join(path, "time_series_covid19_confirmed_global_iso3_regions.csv"))

In [9]:
#this one contains non nulls for Canada
df_recovered_gen = pd.read_csv(os.path.join(path, "time_series_covid19_recovered_global.csv"))

In [10]:
df_confirmed =  pd.read_csv(os.path.join(path, "time_series_covid19_confirmed_global_narrow.csv"))
df_deaths =  pd.read_csv(os.path.join(path, "time_series_covid19_deaths_global_narrow.csv"))
df_recovered =  pd.read_csv(os.path.join(path, "time_series_covid19_recovered_global_narrow.csv"))

C:\Users\Susan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning:

Columns (2,3,5,7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.



In [11]:
df_owid =  pd.read_csv(os.path.join(path, "owid-covid-data.csv"))

In [12]:
df_owid.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,ABW,North America,Aruba,2020-03-13,2.0,2.0,NaN,NaN,0.0,NaN,18.733,18.733,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
1,ABW,North America,Aruba,2020-03-19,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
2,ABW,North America,Aruba,2020-03-20,4.0,2.0,0.286,NaN,0.0,0.0,37.465,18.733,2.676,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
3,ABW,North America,Aruba,2020-03-21,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
4,ABW,North America,Aruba,2020-03-22,NaN,NaN,0.286,NaN,NaN,0.0,NaN,NaN,2.676,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.44,106766.0,584.8,41.2,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN


In [47]:
df_owid[df_owid.iso_code == "MSR"]

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,tests_per_case,positive_rate,tests_units,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
36058,MSR,North America,Montserrat,2020-03-21,1.0,1.0,NaN,NaN,0.0,NaN,200.04,200.04,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.96,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36059,MSR,North America,Montserrat,2020-03-22,1.0,0.0,NaN,NaN,0.0,NaN,200.04,0.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.96,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36060,MSR,North America,Montserrat,2020-03-23,1.0,0.0,NaN,NaN,0.0,NaN,200.04,0.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.96,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36061,MSR,North America,Montserrat,2020-03-24,1.0,0.0,NaN,NaN,0.0,NaN,200.04,0.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.96,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36062,MSR,North America,Montserrat,2020-03-25,1.0,0.0,NaN,NaN,0.0,NaN,200.04,0.00,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.96,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36290,MSR,North America,Montserrat,2020-11-08,13.0,0.0,0.0,1.0,0.0,0.0,2600.52,0.00,0.0,200.04,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36291,MSR,North America,Montserrat,2020-11-09,13.0,0.0,0.0,1.0,0.0,0.0,2600.52,0.00,0.0,200.04,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36292,MSR,North America,Montserrat,2020-11-10,13.0,0.0,0.0,1.0,0.0,0.0,2600.52,0.00,0.0,200.04,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN
36293,MSR,North America,Montserrat,2020-11-11,13.0,0.0,0.0,1.0,0.0,0.0,2600.52,0.00,0.0,200.04,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4999.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.16,NaN


In [13]:
df_owid_sub = df_owid[['iso_code', 'continent', 'location','population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index']]

In [15]:
df_demography = df_owid_sub.groupby("location").first().reset_index()

In [17]:
df_demography.shape

(216, 18)

In [18]:
def drop_first_row(df_conf, df_death, df_recover):
    df_conf = df_conf.iloc[1:, :]
    df_death = df_death.iloc[1:, :]
    df_recover = df_recover.iloc[1:, :]
    return df_conf, df_death, df_recover

In [19]:
def convert_source_column_types(df_conf, df_death, df_recover):
    df_conf["Date"] = pd.to_datetime(df_conf["Date"])
    df_death["Date"] = pd.to_datetime(df_death["Date"])
    df_recover["Date"] = pd.to_datetime(df_recover["Date"])
    return df_conf, df_death, df_recover

In [30]:
# combine data from three datasets into one
def combine_data1(df_conf, df_death, df_recover):
    df_new = df_conf.copy()
    df_new = df_new.rename(columns={"Value": "Confirmed"})
    df_new = pd.merge(df_new, df_death[["Province/State", "Country/Region", "Date", "Value"]], how="left", on=["Province/State", "Country/Region", "Date"])
    df_new = df_new.rename(columns={"Value": "Deaths"})    
    df_new = pd.merge(df_new, df_recover[["Province/State", "Country/Region", "Date", "Value"]], how="left", on=["Province/State","Country/Region", "Date"])
    df_new = df_new.rename(columns={"Value": "Recovered"})        
    return df_new
    

In [21]:
def convert_column_type(df):
    df = df.copy()
    df = df.astype({"Confirmed":int, "Deaths":int, "Recovered":int, #Recovered contain null values for canada
                   "Lat":float, "Long":float, "Date":"datetime64[ns]"})
    return df

In [22]:
def replace_nulls_in_canada(df):
    df.loc[df.Country=="Canada", "Recovered"] = 0
    return df

In [183]:
def modify_country_name(x):
    if (x["Country"] in ['China', 'Denmark', 'France', 'Netherlands', 'United Kingdom']) and not(x["ISO_Code"] in ["CHN", "GBR", "NLD", "DNK", "FRA"]):
        return x["State"] + " (" + x["Country"] + ")"        
    else:
        return x["Country"]

In [193]:
# set the country name at the colony level, ie. places with specific ISO_code
def adjust_colony_names(df1):
    df1 = df1.copy()
    df1["Country"] = df1.apply(modify_country_name, axis=1)   
    return df1

In [204]:
def aggregate_at_country_level(df):
    new_df = df.copy()
    new_df = adjust_colony_names(new_df)
    
    new_df = new_df.groupby(['ISO_Code', 'Country', 'Date']).agg({
        'Confirmed' : np.sum, 
        'Deaths' : np.sum,
        'Recovered' : np.sum, 
        'Lat' : np.mean, 
        'Long' : np.mean,
        'Region_Code': "first",
        "Subregion_Code": "first",
        "Intermeidate_Region_Code": "first"
    }).reset_index()
    return new_df
    

In [196]:
def preprocess_data(df_conf, df_death, df_recover):
    df_conf = df_conf.copy()
    df_death = df_death.copy()
    df_recover = df_recover.copy()
    
    df_conf, df_death, df_recover = drop_first_row(df_conf, df_death, df_recover)
    df_new = combine_data1(df_conf, df_death, df_recover)
    df_new = df_new.rename(columns={"Country/Region":"Country", "Province/State":"State", "ISO 3166-1 Alpha 3-Codes":"ISO_Code",
                                   "Intermediate Region Code": "Intermeidate_Region_Code", 'Region Code':"Region_Code", 
                                    "Sub-region Code":"Subregion_Code"})
    df_new = replace_nulls_in_canada(df_new)    
    df_new = df_new[['ISO_Code', 'Country', 'State', 'Date', 'Confirmed', 'Deaths', 'Recovered',
       'Lat', 'Long', 'Region_Code', 'Subregion_Code', 'Intermeidate_Region_Code']]
    #print(df_new.info())
    df_new = convert_column_type(df_new).sort_values(by=["Country", "Date"])
    df_agg = aggregate_at_country_level(df_new).sort_values(by=["Country", "Date"])
    return df_new, df_agg

In [197]:
# gets canadian recovery data from dataframe and shape it into narrow form so that it can be merged with 
# another dataframe
def get_canada_recover_stats(df):
    df = df.copy()
    df = df[df["Country/Region"]=="Canada"]
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = pd.melt(df, id_vars =['Country/Region']) 
    df = df.rename(columns={'Country/Region': "Country", "variable":"Date", "value":"Recovered"})
    df = df.astype({"Recovered":int, "Date":"datetime64[ns]"})
    return df

In [198]:
# replace blank values for Recovered counts in canada with data from another dataframe
def fix_canadian_recover_counts(df1, df_can1):
    df1 = df1.copy()
    df_can1 = df_can1.copy()
    s = df_can1.set_index(["Country", "Date"])["Recovered"]
    df1.loc[df1["Country"]=="Canada",'Recovered'] = df1[df1["Country"]=="Canada"].set_index(['Country', "Date"]).index.map(s)
    return df1

In [199]:
def merge_with_demography_data(df1, df_demo):
    df1 = df1.copy()
    res = pd.merge(df1, df_demo, how="left", left_on="ISO_Code", right_on="iso_code")
    res = res[['ISO_Code', 'Country', 'continent', 'Date', 'Confirmed', 'Deaths', 'Recovered',
       'Lat', 'Long', 'Region_Code', 'Subregion_Code',
       'Intermeidate_Region_Code', 
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index']]
    return res

### start by preprocessing data

In [205]:
df_processed, df_country = preprocess_data(df_confirmed, df_deaths, df_recovered)

In [206]:
df_processed.shape, df_country.shape

((79355, 12), (63720, 11))

In [207]:
df_country.Country.unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Anguilla (United Kingdom)', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Aruba (Netherlands)', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda (United Kingdom)', 'Bhutan', 'Bolivia',
       'Bonaire, Sint Eustatius and Saba (Netherlands)',
       'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'British Virgin Islands (United Kingdom)', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia',
       'Cameroon', 'Canada', 'Cayman Islands (United Kingdom)',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao (Netherlands)',
       'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 

In [37]:
df_processed[df_processed.Country == "United Kingdom"].groupby("Country").

,ISO_Code,Country,State,Date,Confirmed,Deaths,Recovered,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code
73159,GBR,United Kingdom,NaN,2020-01-22,0,0,0,55.378100,-3.436000,150,154,NaN
73454,AIA,United Kingdom,Anguilla,2020-01-22,0,0,0,18.220600,-63.068600,19,419,29
73749,BMU,United Kingdom,Bermuda,2020-01-22,0,0,0,32.307800,-64.750500,19,21,NaN
74044,VGB,United Kingdom,British Virgin Islands,2020-01-22,0,0,0,18.420700,-64.640000,19,419,29
74339,CYM,United Kingdom,Cayman Islands,2020-01-22,0,0,0,19.313300,-81.254600,19,419,29
...,...,...,...,...,...,...,...,...,...,...,...,...
74635,FLK,United Kingdom,Falkland Islands (Malvinas),2020-11-11,14,0,13,-51.796300,-59.523600,19,419,5
74930,GIB,United Kingdom,Gibraltar,2020-11-11,842,1,712,36.140800,-5.353600,150,39,NaN
75225,IMN,United Kingdom,Isle of Man,2020-11-11,361,25,329,54.236100,-4.548100,150,154,NaN
75520,MSR,United Kingdom,Montserrat,2020-11-11,13,1,12,16.742498,-62.187366,19,419,29


In [208]:
df_country[df_country.Country == "United Kingdom"]

,ISO_Code,Country,Date,Confirmed,Deaths,Recovered,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code
20650,GBR,United Kingdom,2020-01-22,0,0,0,55.3781,-3.436,150,154,NaN
20651,GBR,United Kingdom,2020-01-23,0,0,0,55.3781,-3.436,150,154,NaN
20652,GBR,United Kingdom,2020-01-24,0,0,0,55.3781,-3.436,150,154,NaN
20653,GBR,United Kingdom,2020-01-25,0,0,0,55.3781,-3.436,150,154,NaN
20654,GBR,United Kingdom,2020-01-26,0,0,0,55.3781,-3.436,150,154,NaN
...,...,...,...,...,...,...,...,...,...,...,...
20940,GBR,United Kingdom,2020-11-07,1171441,48888,0,55.3781,-3.436,150,154,NaN
20941,GBR,United Kingdom,2020-11-08,1192013,49044,0,55.3781,-3.436,150,154,NaN
20942,GBR,United Kingdom,2020-11-09,1213363,49238,0,55.3781,-3.436,150,154,NaN
20943,GBR,United Kingdom,2020-11-10,1233775,49770,0,55.3781,-3.436,150,154,NaN


In [209]:
### Canadian data do not contain Recovery counts, so we need to get it from another sourceM

In [210]:
df_can = get_canada_recover_stats(df_recovered_gen)
df_can

,Country,Date,Recovered
0,Canada,2020-01-22,0
1,Canada,2020-01-23,0
2,Canada,2020-01-24,0
3,Canada,2020-01-25,0
4,Canada,2020-01-26,0
...,...,...,...
291,Canada,2020-11-08,218279
292,Canada,2020-11-09,221758
293,Canada,2020-11-10,224912
294,Canada,2020-11-11,226826


In [211]:
df_country.Country.nunique(), df_processed.Country.nunique(), df_confirmed['Country/Region'].nunique(),  df_deaths['Country/Region'].nunique(), df_recovered['Country/Region'].nunique()

(216, 191, 192, 192, 192)

In [212]:
df_processed[~df_processed['Country'].isin(df_country.Country)].Country.unique()

array(['Diamond Princess', 'MS Zaandam'], dtype=object)

In [213]:
## now merge dataset to update Canadian recovery counts

In [214]:
df_country1 = fix_canadian_recover_counts(df_country, df_can)

In [215]:
df_country1[df_country1.Country=="Canada"]

,ISO_Code,Country,Date,Confirmed,Deaths,Recovered,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code
10030,CAN,Canada,2020-01-22,0,0,0,53.212983,-93.290858,19,21,NaN
10031,CAN,Canada,2020-01-23,0,0,0,53.212983,-93.290858,19,21,NaN
10032,CAN,Canada,2020-01-24,0,0,0,53.212983,-93.290858,19,21,NaN
10033,CAN,Canada,2020-01-25,0,0,0,53.212983,-93.290858,19,21,NaN
10034,CAN,Canada,2020-01-26,1,0,0,53.212983,-93.290858,19,21,NaN
...,...,...,...,...,...,...,...,...,...,...,...
10320,CAN,Canada,2020-11-07,263207,10546,216338,53.212983,-93.290858,19,21,NaN
10321,CAN,Canada,2020-11-08,267150,10574,218279,53.212983,-93.290858,19,21,NaN
10322,CAN,Canada,2020-11-09,272021,10619,221758,53.212983,-93.290858,19,21,NaN
10323,CAN,Canada,2020-11-10,276466,10692,224912,53.212983,-93.290858,19,21,NaN


In [216]:
## get additional demography data for each country

In [217]:
df_country2 = merge_with_demography_data(df_country1, df_demography)

In [218]:
df_country2

,ISO_Code,Country,continent,Date,Confirmed,Deaths,Recovered,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Afghanistan,Asia,2020-01-22,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Afghanistan,Asia,2020-01-23,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Afghanistan,Asia,2020-01-24,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Afghanistan,Asia,2020-01-25,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Afghanistan,Asia,2020-01-26,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63715,ZWE,Zimbabwe,Africa,2020-11-07,8498,251,7995,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
63716,ZWE,Zimbabwe,Africa,2020-11-08,8531,253,8005,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
63717,ZWE,Zimbabwe,Africa,2020-11-09,8561,254,8023,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
63718,ZWE,Zimbabwe,Africa,2020-11-10,8610,255,8040,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535


In [219]:
def date_diff(d1, d2):
    return (d1 - d2).days

In [220]:
(df_country2.loc[2, "Date"] - df_country2.loc[1, "Date"]).days

1

In [221]:
date_diff(df_country2.loc[2, "Date"], df_country2.loc[1, "Date"])

1

In [222]:
def add_calculated_columns(df1):
    df1 = df1.copy()
    df1['ActiveCases'] = df1['Confirmed'] - df1['Deaths'] - df1['Recovered']
    df1['CaseFatalityRate'] = df1["Deaths"] / df1["Confirmed"]
    df1['CaseRecoveryRate'] = df1["Recovered"] / df1["Confirmed"]
    df1['DeathVsRecoveryRatio'] = df1["Deaths"] / (df1["Recovered"] + df1["Deaths"])
    return df1

In [223]:
df_country3 = add_calculated_columns(df_country2)

In [225]:
def add_population_adjustd_columns(df1):
    df1 = df1.copy()
    df1["ConfirmedPer1MilPop"] = round(df1["Confirmed"] / df1["population"] * 1000000, 3)
    df1["DeathPer1MilPop"] = round(df1["Deaths"] / df1["population"] * 1000000, 3)
    df1["ActiveCasesPer1MilPop"] = round(df1["ActiveCases"] / df1["population"] * 1000000, 3)    
    return df1

In [226]:
df_country4 = add_population_adjustd_columns(df_country3)

In [227]:
df_country4

,ISO_Code,Country,continent,Date,Confirmed,Deaths,Recovered,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,ActiveCases,CaseFatalityRate,CaseRecoveryRate,DeathVsRecoveryRatio,ConfirmedPer1MilPop,DeathPer1MilPop,ActiveCasesPer1MilPop
0,AFG,Afghanistan,Asia,2020-01-22,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,0,NaN,NaN,NaN,0.000,0.000,0.000
1,AFG,Afghanistan,Asia,2020-01-23,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,0,NaN,NaN,NaN,0.000,0.000,0.000
2,AFG,Afghanistan,Asia,2020-01-24,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,0,NaN,NaN,NaN,0.000,0.000,0.000
3,AFG,Afghanistan,Asia,2020-01-25,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,0,NaN,NaN,NaN,0.000,0.000,0.000
4,AFG,Afghanistan,Asia,2020-01-26,0,0,0,33.939110,67.709953,142,34,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,0,NaN,NaN,NaN,0.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63715,ZWE,Zimbabwe,Africa,2020-11-07,8498,251,7995,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,252,0.029536,0.940810,0.030439,571.758,16.888,16.955
63716,ZWE,Zimbabwe,Africa,2020-11-08,8531,253,8005,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,273,0.029657,0.938343,0.030637,573.978,17.022,18.368
63717,ZWE,Zimbabwe,Africa,2020-11-09,8561,254,8023,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,284,0.029669,0.937157,0.030687,575.997,17.090,19.108
63718,ZWE,Zimbabwe,Africa,2020-11-10,8610,255,8040,-19.015438,29.154857,2,202,14,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,315,0.029617,0.933798,0.030741,579.294,17.157,21.194


In [228]:
def add_aggregate_cols(df):
    df = df.copy()
    df['NewConfirmed'] = (df.groupby("Country")['Confirmed'].apply(lambda x:  x - x.shift()))
    df['NewDeaths'] = (df.groupby("Country")['Deaths'].apply(lambda x:  x - x.shift()))
    df['NewRecovered'] = (df.groupby("Country")['Recovered'].apply(lambda x:  x - x.shift()))
    df["NewConfirmedPer1MilPop"] = round(df["NewConfirmed"] / df["population"] * 1000000, 3)
    df["NewDeathPer1MilPop"] = round(df["NewDeaths"] / df["population"] * 1000000, 3)
    return df

In [229]:
def add_aggregate_cols2(df):
    df = df.copy()
    df['DailyConfirmedGrowthRate'] = (df.groupby("Country")['Confirmed'].apply(lambda x:  (x - x.shift()) / x.shift()))
    df['DailyDeathsGrowthRate'] = (df.groupby("Country")['Deaths'].apply(lambda x:  (x - x.shift()) / x.shift()))
    df['DailyRecoveredGrowthRate'] = (df.groupby("Country")['Recovered'].apply(lambda x:  (x - x.shift()) / x.shift()))
    return df

In [230]:
def add_additional_stats(df):
    df = df.copy()
    
    df["1stConfirmed"] = df["Confirmed"] >= 1
    df["100thConfirmed"] = df["Confirmed"] >= 100
    df["500thConfirmed"] = df["Confirmed"] >= 500    

    df["1stDeath"] = df["Deaths"] >= 1
    df["10thDeath"] = df["Deaths"] >= 10

    df["50thDeath"] = df["Deaths"] >= 50


    df["DaysSince1stDeath"] = df[df["1stDeath"]].groupby("Country").cumcount()
    df["DaysSince10thDeath"] = df[df["10thDeath"]].groupby("Country").cumcount()
    df["DaysSince50thDeath"] = df[df["50thDeath"]].groupby("Country").cumcount()

    df["DaysSince1stCase"] = df[df["1stConfirmed"]].groupby("Country").cumcount()
    df["DaysSince100thCase"] = df[df["100thConfirmed"]].groupby("Country").cumcount()
    df["DaysSince500thCase"] = df[df["500thConfirmed"]].groupby("Country").cumcount()
    
    return df

In [231]:
def reorganise_columns(df):
    df = df.copy()
    col_list = ['population', 'population_density',
       'median_age', 'aged_65_older', 'aged_70_older', 'gdp_per_capita',
       'extreme_poverty', 'cardiovasc_death_rate', 'diabetes_prevalence',
       'female_smokers', 'male_smokers', 'handwashing_facilities',
       'hospital_beds_per_thousand', 'life_expectancy',
       'human_development_index']
    rep_dict = {'population':'Population', 'population_density':'Population_Density',
       'median_age':'Median_Age', 'aged_65_older':'Aged_65+', 'aged_70_older':'Aged_70+', 
       'gdp_per_capita':'Gdp_per_Capita', 'extreme_poverty':'Extreme_Poverty', 'cardiovasc_death_rate':'Cardiovasc_Death_Rate',
       'diabetes_prevalence':'Diabetes_Prevalence', 'female_smokers':'Female_Smokers', 'male_smokers':'Male_Smokers', 
       'handwashing_facilities':'Handwashing_Facilities','hospital_beds_per_thousand':'Hospital_Beds_per_Thousand', 
       'life_expectancy':'Life_Expectancy','human_development_index':'Human_Development_Index'}
    df = df.rename(columns=rep_dict)
    df = df[['ISO_Code', 'Country', 'continent', 'Date', 'Confirmed', 'Deaths','Recovered', 'ActiveCases', 
       'NewConfirmed','NewDeaths', 'NewRecovered', "NewConfirmedPer1MilPop", "NewDeathPer1MilPop",
       'CaseFatalityRate', 'CaseRecoveryRate', 'DeathVsRecoveryRatio', 
       'ConfirmedPer1MilPop', 'DeathPer1MilPop', 'ActiveCasesPer1MilPop', 
       'DailyConfirmedGrowthRate', 'DailyDeathsGrowthRate', 'DailyRecoveredGrowthRate', 
       'DaysSince1stDeath', 'DaysSince10thDeath', 'DaysSince50thDeath',
       'DaysSince1stCase', 'DaysSince100thCase', 'DaysSince500thCase',
       'Population', 'Population_Density',
       'Median_Age', 'Aged_65+', 'Aged_70+', 'Gdp_per_Capita',
       'Extreme_Poverty', 'Cardiovasc_Death_Rate', 'Diabetes_Prevalence',
       'Female_Smokers', 'Male_Smokers', 'Handwashing_Facilities',
       'Hospital_Beds_per_Thousand', 'Life_Expectancy',
       'Human_Development_Index',  '1stConfirmed', '100thConfirmed',
       '500thConfirmed', '1stDeath', '10thDeath', '50thDeath',
       'Lat', 'Long', 'Region_Code', 'Subregion_Code',
       'Intermeidate_Region_Code']]
    return df

In [232]:
df_country5 = add_aggregate_cols(df_country4)

In [233]:
df_country6 = add_aggregate_cols2(df_country5)

In [234]:
df_country7 = add_additional_stats(df_country6)

In [235]:
df_country8 = reorganise_columns(df_country7)

In [236]:
df_country8

,ISO_Code,Country,continent,Date,Confirmed,Deaths,Recovered,ActiveCases,NewConfirmed,NewDeaths,NewRecovered,NewConfirmedPer1MilPop,NewDeathPer1MilPop,CaseFatalityRate,CaseRecoveryRate,DeathVsRecoveryRatio,ConfirmedPer1MilPop,DeathPer1MilPop,ActiveCasesPer1MilPop,DailyConfirmedGrowthRate,DailyDeathsGrowthRate,DailyRecoveredGrowthRate,DaysSince1stDeath,DaysSince10thDeath,DaysSince50thDeath,DaysSince1stCase,DaysSince100thCase,DaysSince500thCase,Population,Population_Density,Median_Age,Aged_65+,Aged_70+,Gdp_per_Capita,Extreme_Poverty,Cardiovasc_Death_Rate,Diabetes_Prevalence,Female_Smokers,Male_Smokers,Handwashing_Facilities,Hospital_Beds_per_Thousand,Life_Expectancy,Human_Development_Index,1stConfirmed,100thConfirmed,500thConfirmed,1stDeath,10thDeath,50thDeath,Lat,Long,Region_Code,Subregion_Code,Intermeidate_Region_Code
0,AFG,Afghanistan,Asia,2020-01-22,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,False,False,False,False,False,False,33.939110,67.709953,142,34,NaN
1,AFG,Afghanistan,Asia,2020-01-23,0,0,0,0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,False,False,False,False,False,False,33.939110,67.709953,142,34,NaN
2,AFG,Afghanistan,Asia,2020-01-24,0,0,0,0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,False,False,False,False,False,False,33.939110,67.709953,142,34,NaN
3,AFG,Afghanistan,Asia,2020-01-25,0,0,0,0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,False,False,False,False,False,False,33.939110,67.709953,142,34,NaN
4,AFG,Afghanistan,Asia,2020-01-26,0,0,0,0,0.0,0.0,0.0,0.000,0.000,NaN,NaN,NaN,0.000,0.000,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38928341.0,54.422,18.6,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498,False,False,False,False,False,False,33.939110,67.709953,142,34,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63715,ZWE,Zimbabwe,Africa,2020-11-07,8498,251,7995,252,27.0,1.0,12.0,1.817,0.067,0.029536,0.940810,0.030439,571.758,16.888,16.955,0.003187,0.004000,0.001503,229.0,120.0,100.0,232.0,164.0,138.0,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,True,True,True,True,True,True,-19.015438,29.154857,2,202,14
63716,ZWE,Zimbabwe,Africa,2020-11-08,8531,253,8005,273,33.0,2.0,10.0,2.220,0.135,0.029657,0.938343,0.030637,573.978,17.022,18.368,0.003883,0.007968,0.001251,230.0,121.0,101.0,233.0,165.0,139.0,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,True,True,True,True,True,True,-19.015438,29.154857,2,202,14
63717,ZWE,Zimbabwe,Africa,2020-11-09,8561,254,8023,284,30.0,1.0,18.0,2.018,0.067,0.029669,0.937157,0.030687,575.997,17.090,19.108,0.003517,0.003953,0.002249,231.0,122.0,102.0,234.0,166.0,140.0,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,True,True,True,True,True,True,-19.015438,29.154857,2,202,14
63718,ZWE,Zimbabwe,Africa,2020-11-10,8610,255,8040,315,49.0,1.0,17.0,3.297,0.067,0.029617,0.933798,0.030741,579.294,17.157,21.194,0.005724,0.003937,0.002119,232.0,123.0,103.0,235.0,167.0,141.0,14862927.0,42.729,19.6,2.822,1.882,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535,True,True,True,True,True,True,-19.015438,29.154857,2,202,14


In [237]:
df_country8.columns

Index(['ISO_Code', 'Country', 'continent', 'Date', 'Confirmed', 'Deaths',
       'Recovered', 'ActiveCases', 'NewConfirmed', 'NewDeaths', 'NewRecovered',
       'NewConfirmedPer1MilPop', 'NewDeathPer1MilPop', 'CaseFatalityRate',
       'CaseRecoveryRate', 'DeathVsRecoveryRatio', 'ConfirmedPer1MilPop',
       'DeathPer1MilPop', 'ActiveCasesPer1MilPop', 'DailyConfirmedGrowthRate',
       'DailyDeathsGrowthRate', 'DailyRecoveredGrowthRate',
       'DaysSince1stDeath', 'DaysSince10thDeath', 'DaysSince50thDeath',
       'DaysSince1stCase', 'DaysSince100thCase', 'DaysSince500thCase',
       'Population', 'Population_Density', 'Median_Age', 'Aged_65+',
       'Aged_70+', 'Gdp_per_Capita', 'Extreme_Poverty',
       'Cardiovasc_Death_Rate', 'Diabetes_Prevalence', 'Female_Smokers',
       'Male_Smokers', 'Handwashing_Facilities', 'Hospital_Beds_per_Thousand',
       'Life_Expectancy', 'Human_Development_Index', '1stConfirmed',
       '100thConfirmed', '500thConfirmed', '1stDeath', '10thDeath',
 

In [238]:
df_country8.isna().sum()

ISO_Code                          0
Country                           0
continent                      2950
Date                              0
Confirmed                         0
Deaths                            0
Recovered                         0
ActiveCases                       0
NewConfirmed                    216
NewDeaths                       216
NewRecovered                    216
NewConfirmedPer1MilPop         3156
NewDeathPer1MilPop             3156
CaseFatalityRate              10168
CaseRecoveryRate              10168
DeathVsRecoveryRatio          13047
ConfirmedPer1MilPop            2950
DeathPer1MilPop                2950
ActiveCasesPer1MilPop          2950
DailyConfirmedGrowthRate      10175
DailyDeathsGrowthRate         20562
DailyRecoveredGrowthRate      14461
DaysSince1stDeath             20566
DaysSince10thDeath            29833
DaysSince50thDeath            38767
DaysSince1stCase              10168
DaysSince100thCase            22287
DaysSince500thCase          

In [239]:
df_country8.to_csv(os.path.join(path, "covid_processed.csv"))

### some visualisation

In [240]:
def get_infection_info(db, country):
    df = df.copy()
    inf_info = db
    country_info = inf_info[inf_info["Country"]==country]      
    return country_info

In [249]:
country_widget = widgets.Dropdown(
    options=df_country4["Country"].unique(),
    value=["Australia"]
    
)

TraitError: Invalid selection: value not found

In [242]:
countries_widget1 = widgets.SelectMultiple(
    options=sorted(df_country4["Country"].unique()),
    value=["Portugal"]
)

In [243]:
def get_country_info1(db, countries):    
    inf_info = db.copy()
    country_info = inf_info[inf_info["Country"].isin(countries)]      
    return country_info#.iloc[:, [0,1,2,3,4,5,-3, -2, -1]]

In [244]:
cf.set_config_file(sharing='public',theme='pearl',offline=False)
cf.go_offline()

In [247]:
topic_widget = widgets.SelectMultiple(options = ["Confirmed", "Deaths", "Recovered"], value=['Confirmed'])

def plot_infection_info(db, country, col):
    cols = list(col)
    inf_info = db.copy()
    country_info = inf_info[inf_info["Country"]==country]
    country_info.iplot(kind='scatter',x="Date", y=cols, title=country.upper(),mode='lines+markers', size=5)  
#    return country_info

interact(lambda country, topic: plot_infection_info(df_country4, country, topic), 
        country = country_widget, topic = topic_widget)

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…

<function __main__.<lambda>(country, topic)>

In [246]:
interact(lambda countries: get_country_info1(df_country4, countries), countries = countries_widget1)

interactive(children=(SelectMultiple(description='countries', index=(157,), options=('Afghanistan', 'Albania',…

<function __main__.<lambda>(countries)>